In [54]:
# behave like a normal kernel plz
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# plot inline
%matplotlib inline

In [55]:
import pandas as pd
gene2pubmed = pd.read_table("data/genbank-data/gene2pubmed", sep = "\t")
list(gene2pubmed) # hmm.. the weird # in column header is going to come to bite me if I don't replace it right away !

gene2pubmed.rename(columns={'#tax_id':'tax_id'}, inplace=True)
list(gene2pubmed)

gene2pubmed.head()


['#tax_id', 'GeneID', 'PubMed_ID']

['tax_id', 'GeneID', 'PubMed_ID']

,tax_id,GeneID,PubMed_ID
0,9,1246500,9873079
1,9,1246501,9873079
2,9,1246502,9812361
3,9,1246502,9873079
4,9,1246503,9873079


## Get list of taxid's to (a) get their lineage; (b) Restrict the list to bacteria and finally (c) subset gene2pubmed table to bacteria only entries

In [10]:
txidOnly = gene2pubmed.tax_id.unique()
type(txidOnly)
txidOnly[0:5]
len(txidOnly)

numpy.ndarray

array([  9,  33, 139, 140, 158])

13950

In [74]:
txidtest = txidOnly[0:10000]

In [155]:
import sqlite3
con = sqlite3.connect("NameNode.sqlite")   
Dummymaster = pd.DataFrame(columns=("species","genus","family","order","class","phylum","superkingdom","norank"))
count = 0
#txidtest = txidOnly[4948:5000]
#for tid in txidtest:
for tid in txidOnly:
    ntid = int(tid)
    
    #print(tid)
    df = pd.read_sql_query("SELECT tax_id, name_txt,parent_tax_id, rank FROM NcbiNameNode WHERE tax_id == ? AND name_class == 'scientific name'", con=con, params = (ntid,))
    if df.size == 0:
        print(str(tid) + " check !!")
        count += 1
        continue


175128 check !!
618424 check !!
618637 check !!
675577 check !!
707310 check !!
886957 check !!
886958 check !!
886959 check !!
886960 check !!
886961 check !!
886963 check !!
886966 check !!
886967 check !!
886968 check !!
945028 check !!
1263171 check !!
1333989 check !!
1347737 check !!
1347739 check !!
1351828 check !!
1513278 check !!
1548719 check !!
1565112 check !!
1661584 check !!
1671078 check !!
1671114 check !!
1678146 check !!
1685502 check !!
1715887 check !!
1715890 check !!
1715891 check !!
1715894 check !!
1715900 check !!
1715998 check !!
1715999 check !!
1716000 check !!
1716002 check !!
1716003 check !!
1716004 check !!
1716006 check !!
1716010 check !!
1716011 check !!
1716012 check !!
1716013 check !!
1716014 check !!
1716016 check !!
1716017 check !!
1716018 check !!
1716019 check !!
1716020 check !!
1716021 check !!
1716022 check !!
1716023 check !!
1716024 check !!
1716025 check !!
1716026 check !!
1716027 check !!
1716028 check !!
1716029 check !!
1716030 chec

In [89]:
import sqlite3
con = sqlite3.connect("NameNode.sqlite")   
Dummymaster = pd.DataFrame(columns=("species","genus","family","order","class","phylum","superkingdom","norank"))
count = 0
#txidtest = txidOnly[4948:5000]
#for tid in txidtest:
for tid in txidOnly:
    ntid = int(tid)
    
    #print(tid)
    df = pd.read_sql_query("SELECT tax_id, name_txt,parent_tax_id, rank FROM NcbiNameNode WHERE tax_id == ? AND name_class == 'scientific name'", con=con, params = (ntid,))
    if df.size == 0:
        #print(str(tid) + " check !!")
        count += 1
        continue
    classi = {'taxid': tid,'species': 'unclassified','genus': 'unclassified','family': 'unclassified','order': 'unclassified','class': 'unclassified','phylum': 'unclassified','superkingdom': 'unclassified','norank': 'unclassified'}
    level = df['rank'][0]
    parent = df['parent_tax_id'][0]
    sciname = df['name_txt'][0]
    taxid = int(df['tax_id'][0])

    while parent != 1:
        #print(taxid)
        con = sqlite3.connect("NameNode.sqlite")
        df = pd.read_sql_query("SELECT tax_id, name_txt,parent_tax_id, rank FROM NcbiNameNode WHERE tax_id == ? AND name_class == 'scientific name'", con = con, params=(taxid,))
        #con.close()
        #df
        
        level = df['rank'][0]
        parent = df['parent_tax_id'][0]
        sciname = df['name_txt'][0]
        #tax_id = df['tax_id'][0]
        taxid = int(df['parent_tax_id'][0])
        classi[level] = sciname
        #classi
    # df = pd.DataFrame.from_dict(sales)
    temp = pd.DataFrame.from_dict([classi])
    temp = temp[['taxid','species','genus','family','order','class','phylum','superkingdom','norank']]
    #temp
    #Dummymaster = Dummymaster.append(temp)
    Dummymaster = pd.concat([Dummymaster,temp],axis=0,join='outer')
    #Dummymaster
    

con.close()
Dummymaster = Dummymaster.reset_index()
Dummymaster[['taxid']] = Dummymaster[['taxid']].astype(int)
#list(Dummymaster)
Dummymaster= Dummymaster[['taxid','species','genus','family','order','class','phylum','superkingdom','norank']] 
print("number of unidentified taxid in db = " + str(count) )

Dummymaster.head()
Dummymaster.shape


number of unidentified taxid in db = 1571


,taxid,species,genus,family,order,class,phylum,superkingdom,norank
0,9,Buchnera aphidicola,Buchnera,Enterobacteriaceae,Enterobacteriales,Gammaproteobacteria,Proteobacteria,Bacteria,cellular organisms
1,33,Myxococcus fulvus,Myxococcus,Myxococcaceae,Myxococcales,Deltaproteobacteria,Proteobacteria,Bacteria,cellular organisms
2,139,Borreliella burgdorferi,Borreliella,Borreliaceae,Spirochaetales,Spirochaetia,Spirochaetes,Bacteria,cellular organisms
3,140,Borrelia hermsii,Borrelia,Borreliaceae,Spirochaetales,Spirochaetia,Spirochaetes,Bacteria,cellular organisms
4,158,Treponema denticola,Treponema,Spirochaetaceae,Spirochaetales,Spirochaetia,Spirochaetes,Bacteria,cellular organisms


(12379, 9)

## Identifying bacteria only

In [91]:
BacteriaOnly = Dummymaster.loc[(Dummymaster.superkingdom == 'Bacteria')]
BacteriaOnly.head()
BacteriaOnly.shape

,taxid,species,genus,family,order,class,phylum,superkingdom,norank
0,9,Buchnera aphidicola,Buchnera,Enterobacteriaceae,Enterobacteriales,Gammaproteobacteria,Proteobacteria,Bacteria,cellular organisms
1,33,Myxococcus fulvus,Myxococcus,Myxococcaceae,Myxococcales,Deltaproteobacteria,Proteobacteria,Bacteria,cellular organisms
2,139,Borreliella burgdorferi,Borreliella,Borreliaceae,Spirochaetales,Spirochaetia,Spirochaetes,Bacteria,cellular organisms
3,140,Borrelia hermsii,Borrelia,Borreliaceae,Spirochaetales,Spirochaetia,Spirochaetes,Bacteria,cellular organisms
4,158,Treponema denticola,Treponema,Spirochaetaceae,Spirochaetales,Spirochaetia,Spirochaetes,Bacteria,cellular organisms


(1039, 9)

In [104]:
# pd.merge(df1, df2, on='column_name', how='inner')
BacteriaG2p= pd.merge(BacteriaOnly,gene2pubmed, left_on='taxid', right_on='tax_id', how='inner')
BacteriaG2p = BacteriaG2p.drop(['taxid'], axis = 1)
BacteriaG2p.head()
BacteriaG2p.shape

,species,genus,family,order,class,phylum,superkingdom,norank,tax_id,GeneID,PubMed_ID
0,Buchnera aphidicola,Buchnera,Enterobacteriaceae,Enterobacteriales,Gammaproteobacteria,Proteobacteria,Bacteria,cellular organisms,9,1246500,9873079
1,Buchnera aphidicola,Buchnera,Enterobacteriaceae,Enterobacteriales,Gammaproteobacteria,Proteobacteria,Bacteria,cellular organisms,9,1246501,9873079
2,Buchnera aphidicola,Buchnera,Enterobacteriaceae,Enterobacteriales,Gammaproteobacteria,Proteobacteria,Bacteria,cellular organisms,9,1246502,9812361
3,Buchnera aphidicola,Buchnera,Enterobacteriaceae,Enterobacteriales,Gammaproteobacteria,Proteobacteria,Bacteria,cellular organisms,9,1246502,9873079
4,Buchnera aphidicola,Buchnera,Enterobacteriaceae,Enterobacteriales,Gammaproteobacteria,Proteobacteria,Bacteria,cellular organisms,9,1246503,9873079


(2102509, 11)

To save myself the pain of searching through gene descriptions manually, lets also upload the table of gene info

In [99]:
geneinfo = pd.read_table("data/genbank-data/gene_info_cut1-3_9-10", sep = "\t")
list(geneinfo) # darnit # in column header again !

geneinfo.rename(columns={'#tax_id':'tax_id'}, inplace=True)
list(geneinfo)

geneinfo.head()

['#tax_id', 'GeneID', 'Symbol', 'description', 'type_of_gene']

In [105]:
BacteriaG2p_geneinfo = pd.merge(BacteriaG2p,geneinfo, left_on='GeneID', right_on='GeneID', how='inner')
BacteriaG2p_geneinfo.shape

(2102508, 15)

## Most studied genes in Bacteria

In [124]:
BacteriaG2p_gene_gp_pubmedCount = BacteriaG2p_geneinfo.groupby('GeneID', as_index=False)['PubMed_ID'].agg({'PubMed_pubs': "count",'PubMed_uniqpubs': "nunique"}) # because pandas preserves the row index and we need it reset to be sorted
#BacteriaG2p_gene_gp_pubmedCount.head()
BacteriaG2p_gene_gp_pubmedCount2 = pd.merge(BacteriaG2p_gene_gp_pubmedCount, BacteriaG2p_geneinfo[['GeneID','Symbol','description']], on= 'GeneID', how='left')
#BacteriaG2p_gene_gp_pubmedCount2.head()
#BacteriaG2p_gene_gp_pubmedCount2.shape

BacteriaG2p_gene_gp_pubmedCount2 = BacteriaG2p_gene_gp_pubmedCount2.drop_duplicates()
#BacteriaG2p_gene_gp_pubmedCount2.head()
BacteriaG2p_gene_gp_pubmedCount2 = BacteriaG2p_gene_gp_pubmedCount2.reset_index(drop=True)
#BacteriaG2p_gene_gp_pubmedCount2.head()
#BacteriaG2p_gene_gp_pubmedCount2.shape

BacteriaG2p_gene_gp_pubmedCount2.sort_values('PubMed_pubs', ascending=False).head(n=10)
BacteriaG2p_gene_gp_pubmedCount2.sort_values('PubMed_uniqpubs', ascending=False).head(n=10)


BacteriaG2p_gene_gp_pubmedCount2.sort_values('PubMed_pubs', ascending=False).tail(n=10)
BacteriaG2p_gene_gp_pubmedCount2.sort_values('PubMed_uniqpubs', ascending=False).tail(n=10)

,GeneID,PubMed_pubs,PubMed_uniqpubs,Symbol,description
42993,948466,697,697,rrsB,16S ribosomal RNA of rrnB operon
42322,947777,678,678,rrsD,16S ribosomal RNA of rrnD operon
41642,947071,677,677,rrsG,16S ribosomal RNA of rrnG operon
39536,944897,677,677,rrsH,16S ribosomal RNA of rrnH operon
43036,948511,677,677,rrsE,16S ribosomal RNA of rrnE operon
42803,948270,676,676,rrsC,16S ribosomal RNA of rrnC operon
43000,948473,473,473,rrlB,23S ribosomal RNA of rrnB operon
42134,947585,457,457,rrlG,23S ribosomal RNA of rrnG operon
43034,948509,456,456,rrlE,23S ribosomal RNA of rrnE operon
42318,947773,454,454,rrlD,23S ribosomal RNA of rrnD operon


,GeneID,PubMed_pubs,PubMed_uniqpubs,Symbol,description
42993,948466,697,697,rrsB,16S ribosomal RNA of rrnB operon
42322,947777,678,678,rrsD,16S ribosomal RNA of rrnD operon
41642,947071,677,677,rrsG,16S ribosomal RNA of rrnG operon
39536,944897,677,677,rrsH,16S ribosomal RNA of rrnH operon
43036,948511,677,677,rrsE,16S ribosomal RNA of rrnE operon
42803,948270,676,676,rrsC,16S ribosomal RNA of rrnC operon
43000,948473,473,473,rrlB,23S ribosomal RNA of rrnB operon
42134,947585,457,457,rrlG,23S ribosomal RNA of rrnG operon
43034,948509,456,456,rrlE,23S ribosomal RNA of rrnE operon
42318,947773,454,454,rrlD,23S ribosomal RNA of rrnD operon


,GeneID,PubMed_pubs,PubMed_uniqpubs,Symbol,description
647507,23389632,1,1,SMDB11_RS16085,PTS N-acetylgalactosamine transporter subunit IIB
647506,23389631,1,1,SMDB11_RS16080,PTS acetylgalactosamine transporter subunit IIC
647505,23389630,1,1,SMDB11_RS16075,PTS N-acetylgalactosamine transporter subunit IID
647504,23389629,1,1,SMDB11_RS16070,PTS N-acetylgalactosamine transporter subunit IIA
647503,23389628,1,1,SMDB11_RS16065,ABC transporter ATP-binding protein
647502,23389627,1,1,SMDB11_RS16060,hypothetical protein
647501,23389626,1,1,SMDB11_RS16055,DsbA family protein
647500,23389625,1,1,SMDB11_RS16050,MBL fold metallo-hydrolase
647499,23389624,1,1,SMDB11_RS16045,LysR family transcriptional regulator
1608631,36020782,1,1,GY21_RS16245,mannitol dehydrogenase family protein


,GeneID,PubMed_pubs,PubMed_uniqpubs,Symbol,description
647507,23389632,1,1,SMDB11_RS16085,PTS N-acetylgalactosamine transporter subunit IIB
647506,23389631,1,1,SMDB11_RS16080,PTS acetylgalactosamine transporter subunit IIC
647505,23389630,1,1,SMDB11_RS16075,PTS N-acetylgalactosamine transporter subunit IID
647504,23389629,1,1,SMDB11_RS16070,PTS N-acetylgalactosamine transporter subunit IIA
647503,23389628,1,1,SMDB11_RS16065,ABC transporter ATP-binding protein
647502,23389627,1,1,SMDB11_RS16060,hypothetical protein
647501,23389626,1,1,SMDB11_RS16055,DsbA family protein
647500,23389625,1,1,SMDB11_RS16050,MBL fold metallo-hydrolase
647499,23389624,1,1,SMDB11_RS16045,LysR family transcriptional regulator
1608631,36020782,1,1,GY21_RS16245,mannitol dehydrogenase family protein


In [121]:
# What types of gene descriptions do we have
BacteriaG2p_geneinfo.type_of_gene.unique()

array(['protein-coding', 'other', 'rRNA', 'pseudo', 'ncRNA', 'tRNA',
       'miscRNA'], dtype=object)

#### Lets see what non-rRNA genes or 'protein-coding' genes are most studied

In [125]:
BacteriaG2p_gene_gp_pubmedCount2_coding = pd.merge(BacteriaG2p_gene_gp_pubmedCount, BacteriaG2p_geneinfo[['GeneID','Symbol','description','type_of_gene']], on= 'GeneID', how='left')
# subset protein-coding only
# df.loc[df['column_name'] == some_value]
BacteriaG2p_gene_gp_pubmedCount2_coding = BacteriaG2p_gene_gp_pubmedCount2_coding.loc[BacteriaG2p_gene_gp_pubmedCount2_coding['type_of_gene'] == 'protein-coding']
BacteriaG2p_gene_gp_pubmedCount2_coding = BacteriaG2p_gene_gp_pubmedCount2_coding.drop_duplicates()
BacteriaG2p_gene_gp_pubmedCount2_coding = BacteriaG2p_gene_gp_pubmedCount2_coding.reset_index(drop=True)

BacteriaG2p_gene_gp_pubmedCount2_coding.sort_values('PubMed_pubs', ascending=False).head(n=10)
BacteriaG2p_gene_gp_pubmedCount2_coding.sort_values('PubMed_uniqpubs', ascending=False).head(n=10)


BacteriaG2p_gene_gp_pubmedCount2_coding.sort_values('PubMed_pubs', ascending=False).tail(n=10)
BacteriaG2p_gene_gp_pubmedCount2_coding.sort_values('PubMed_uniqpubs', ascending=False).tail(n=10)

,GeneID,PubMed_pubs,PubMed_uniqpubs,Symbol,description,type_of_gene
41594,947170,418,418,recA,DNA recombination and repair protein; ssDNA-de...,protein-coding
42798,948488,333,333,rpoB,"RNA polymerase, beta subunit",protein-coding
42558,948217,310,310,dnaA,chromosomal replication initiator protein DnaA...,protein-coding
39361,944750,309,309,dnaK,"chaperone Hsp70, with co-chaperone DnaJ",protein-coding
42228,947867,309,309,crp,"cAMP-activated global transcription factor, me...",protein-coding
39395,944786,297,297,ftsZ,GTP-binding tubulin-like cell division protein,protein-coding
41950,947567,291,291,rpoD,"RNA polymerase, sigma 70 (sigma D) factor",protein-coding
41629,947210,287,287,rpoS,"RNA polymerase, sigma S (sigma 38) factor",protein-coding
42797,948487,266,266,rpoC,"RNA polymerase, beta prime subunit",protein-coding
40341,945829,264,264,hns,global DNA-binding transcriptional dual regula...,protein-coding


,GeneID,PubMed_pubs,PubMed_uniqpubs,Symbol,description,type_of_gene
41594,947170,418,418,recA,DNA recombination and repair protein; ssDNA-de...,protein-coding
42798,948488,333,333,rpoB,"RNA polymerase, beta subunit",protein-coding
42558,948217,310,310,dnaA,chromosomal replication initiator protein DnaA...,protein-coding
39361,944750,309,309,dnaK,"chaperone Hsp70, with co-chaperone DnaJ",protein-coding
42228,947867,309,309,crp,"cAMP-activated global transcription factor, me...",protein-coding
39395,944786,297,297,ftsZ,GTP-binding tubulin-like cell division protein,protein-coding
41950,947567,291,291,rpoD,"RNA polymerase, sigma 70 (sigma D) factor",protein-coding
41629,947210,287,287,rpoS,"RNA polymerase, sigma S (sigma 38) factor",protein-coding
42797,948487,266,266,rpoC,"RNA polymerase, beta prime subunit",protein-coding
40341,945829,264,264,hns,global DNA-binding transcriptional dual regula...,protein-coding


,GeneID,PubMed_pubs,PubMed_uniqpubs,Symbol,description,type_of_gene
647063,23389817,1,1,SMDB11_RS17010,hypothetical protein,protein-coding
647062,23389816,1,1,SMDB11_RS17005,molybdopterin-guanine dinucleotide biosynthesi...,protein-coding
647061,23389815,1,1,SMDB11_RS17000,aldehyde oxidase,protein-coding
647060,23389814,1,1,SMDB11_RS16995,(2Fe-2S)-binding protein,protein-coding
647059,23389813,1,1,SMDB11_RS16990,cytochrome Cbb3,protein-coding
647058,23389812,1,1,SMDB11_RS16985,aromatic amino acid transporter,protein-coding
647057,23389811,1,1,SMDB11_RS16980,transcriptional regulator PdhR,protein-coding
647056,23389810,1,1,SMDB11_RS16975,"pyruvate dehydrogenase (acetyl-transferring), ...",protein-coding
647055,23389809,1,1,aceF,pyruvate dehydrogenase complex dihydrolipoylly...,protein-coding
1608007,36020782,1,1,GY21_RS16245,mannitol dehydrogenase family protein,protein-coding


,GeneID,PubMed_pubs,PubMed_uniqpubs,Symbol,description,type_of_gene
647063,23389817,1,1,SMDB11_RS17010,hypothetical protein,protein-coding
647062,23389816,1,1,SMDB11_RS17005,molybdopterin-guanine dinucleotide biosynthesi...,protein-coding
647061,23389815,1,1,SMDB11_RS17000,aldehyde oxidase,protein-coding
647060,23389814,1,1,SMDB11_RS16995,(2Fe-2S)-binding protein,protein-coding
647059,23389813,1,1,SMDB11_RS16990,cytochrome Cbb3,protein-coding
647058,23389812,1,1,SMDB11_RS16985,aromatic amino acid transporter,protein-coding
647057,23389811,1,1,SMDB11_RS16980,transcriptional regulator PdhR,protein-coding
647056,23389810,1,1,SMDB11_RS16975,"pyruvate dehydrogenase (acetyl-transferring), ...",protein-coding
647055,23389809,1,1,aceF,pyruvate dehydrogenase complex dihydrolipoylly...,protein-coding
1608007,36020782,1,1,GY21_RS16245,mannitol dehydrogenase family protein,protein-coding


#### Species ?

In [146]:
import numpy as np
#BacteriaG2p.head(n=2)

Bacteria_spp_pubmeduniqCount = BacteriaG2p.groupby('tax_id', as_index=False)['PubMed_ID'].agg({'Pubmed_count': "count",'Pubmed_uniqcount': 'nunique'}) # because pandas preserves the row index and we need it reset to be sorted
#Bacteria_pubmedCount.head()

spp_pubmeduniqCountspp =  Bacteria_spp_pubmeduniqCount[['tax_id','Pubmed_uniqcount']].sort_values('Pubmed_uniqcount', ascending=False)
spp_pubmeduniqCountspp['Pubmed_uniqCount_percent'] = (spp_pubmeduniqCountspp['Pubmed_uniqcount']/spp_pubmeduniqCountspp['Pubmed_uniqcount'].sum())*100
spp_pubmeduniqCountspp.head(n=10)
#totalpubmeduniqCountspp.tail(n=10)

spp_pubmeduniqCountspp2 = pd.merge(spp_pubmeduniqCountspp,BacteriaG2p[['tax_id','species']], on='tax_id', how='left')
spp_pubmeduniqCountspp2 = spp_pubmeduniqCountspp2.drop_duplicates()
spp_pubmeduniqCountspp2 = spp_pubmeduniqCountspp2.reset_index(drop=True)
spp_pubmeduniqCountspp2.head(n=10)
spp_pubmeduniqCountspp2.tail(n=10)

,species,genus,family,order,class,phylum,superkingdom,norank,tax_id,GeneID,PubMed_ID
0,Buchnera aphidicola,Buchnera,Enterobacteriaceae,Enterobacteriales,Gammaproteobacteria,Proteobacteria,Bacteria,cellular organisms,9,1246500,9873079
1,Buchnera aphidicola,Buchnera,Enterobacteriaceae,Enterobacteriales,Gammaproteobacteria,Proteobacteria,Bacteria,cellular organisms,9,1246501,9873079


,tax_id,Pubmed_uniqcount,Pubmed_uniqCount_percent
721,511145,35636,53.550122
643,386585,3204,4.814642
490,224308,2395,3.598960
363,83332,1968,2.957308
383,99287,1557,2.339700
473,208964,1519,2.282597
47,562,974,1.463627
572,300852,719,1.080439
460,198214,668,1.003802
384,100226,656,0.985769


,tax_id,Pubmed_uniqcount,Pubmed_uniqCount_percent,species
0,511145,35636,53.550122,Escherichia coli
1,386585,3204,4.814642,Escherichia coli
2,224308,2395,3.598960,Bacillus subtilis
3,83332,1968,2.957308,Mycobacterium tuberculosis
4,99287,1557,2.339700,Salmonella enterica
5,208964,1519,2.282597,Pseudomonas aeruginosa
6,562,974,1.463627,Escherichia coli
7,300852,719,1.080439,Thermus thermophilus
8,198214,668,1.003802,Shigella flexneri
9,100226,656,0.985769,Streptomyces coelicolor


,tax_id,Pubmed_uniqcount,Pubmed_uniqCount_percent,species
1029,1464,1,0.001503,Paenibacillus larvae
1030,360016,1,0.001503,Rahnella sp. 'WMR15'
1031,359132,1,0.001503,Bifidobacterium sp. A24
1032,1478,1,0.001503,Bacillus simplex
1033,357244,1,0.001503,Orientia tsutsugamushi
1034,356621,1,0.001503,Salmonella enterica
1035,355276,1,0.001503,Leptospira borgpetersenii
1036,349741,1,0.001503,Akkermansia muciniphila
1037,349519,1,0.001503,Leuconostoc citreum
1038,1771959,1,0.001503,Arthrobacter sp. ATCC 21022


In [154]:
## 349741 #Akkermansia muciniphila

#Akkermansia_muciniphila = BacteriaG2p.loc[(BacteriaG2p.tax_id == 349741)]
#Akkermansia_muciniphila = Akkermansia_muciniphila.drop_duplicates()
#Akkermansia_muciniphila = Akkermansia_muciniphila.reset_index(drop=True)
#Akkermansia_muciniphila

#### Genus

In [147]:
import numpy as np
#BacteriaG2p.head(n=2)

Bacteria_genus_pubmeduniqCount = BacteriaG2p.groupby('genus', as_index=False)['PubMed_ID'].agg({'Pubmed_count': "count",'Pubmed_uniqcount': 'nunique'}) # because pandas preserves the row index and we need it reset to be sorted
#Bacteria_pubmedCount.head()

genus_pubmeduniqCountspp =  Bacteria_genus_pubmeduniqCount[['genus','Pubmed_uniqcount']].sort_values('Pubmed_uniqcount', ascending=False)
genus_pubmeduniqCountspp['Pubmed_uniqCount_percent'] = (genus_pubmeduniqCountspp['Pubmed_uniqcount']/genus_pubmeduniqCountspp['Pubmed_uniqcount'].sum())*100
genus_pubmeduniqCountspp.head(n=10)
#totalpubmeduniqCountspp.tail(n=10)

#genus_pubmeduniqCountspp2 = pd.merge(genus_pubmeduniqCountspp,BacteriaG2p[['genus','genus']], on='tax_id', how='left')
#genus_pubmeduniqCountspp2 = genus_pubmeduniqCountspp2.drop_duplicates()
#genus_pubmeduniqCountspp2 = genus_pubmeduniqCountspp2.reset_index(drop=True)
#genus_pubmeduniqCountspp2.head(n=10)
#genus_pubmeduniqCountspp2.tail(n=10)

,species,genus,family,order,class,phylum,superkingdom,norank,tax_id,GeneID,PubMed_ID
0,Buchnera aphidicola,Buchnera,Enterobacteriaceae,Enterobacteriales,Gammaproteobacteria,Proteobacteria,Bacteria,cellular organisms,9,1246500,9873079
1,Buchnera aphidicola,Buchnera,Enterobacteriaceae,Enterobacteriales,Gammaproteobacteria,Proteobacteria,Bacteria,cellular organisms,9,1246501,9873079


,genus,Pubmed_uniqcount,Pubmed_uniqCount_percent
76,Escherichia,37141,60.194159
22,Bacillus,3039,4.925286
139,Mycobacterium,2442,3.957732
179,Pseudomonas,2326,3.769732
199,Salmonella,1847,2.993420
215,Streptococcus,965,1.563969
212,Staphylococcus,848,1.374348
216,Streptomyces,805,1.304658
203,Shigella,765,1.239830
230,Thermus,720,1.166899
